In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow_addons.optimizers import RectifiedAdam

import sys
import torch
from sklearn.model_selection import train_test_split
from network_tools import *
from hnet import *
from RHCnet_Custom_dataset import CustomDataset, TF_Dataloader, ad_TF_Dataloader_Norm
%pylab inline

# path to the pre-trained model
PRE_TRAINED_MODEL_PATH = './ecg_supervised_416.h5'

c:\Anaconda\envs\tf_keras\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Anaconda\envs\tf_keras\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Adult

In [6]:
adult_train = pd.read_csv('./dataset/ECG_adult_age_train_splited.csv')
adult_valid = pd.read_csv('./dataset/ECG_adult_age_valid_splited.csv')
new_adult_train = pd.read_csv('./dataset/new_train_adult.csv')
adult_train_ = pd.concat([adult_train,new_adult_train])
adult_train_.reset_index(drop=True,inplace=True)

In [10]:
# For classification Version

# sampled_df = adult_train.groupby('seped_AGE').apply(lambda x: x.sample(n = 2000, random_state=42)).reset_index(drop=True)
# sampled_df.drop('seped_AGE',axis=1,inplace=True)
# adult_train.drop('AGE',axis=1,inplace=True)
# adult_train.columns = ['FILENAME','GENDER','AGE']
# adult_train['AGE'] =  adult_train['AGE'] - 2

# category = 7
# im = np.eye(7)
# train_label = im[train_label]
# valid_label = im[valid_label]

In [11]:
# adult_file_path = './dataset/ECG_adult_numpy_train/'
# adult_train_data, adult_valid_data = train_test_split(adult_train,shuffle=True, random_state=42, test_size=0.2, stratify = adult_train['seped_AGE'])
# adult_train_data.drop('seped_AGE',axis=1,inplace=True)
# adult_valid_data.drop('seped_AGE',axis=1,inplace=True)
# adult_train_data.reset_index(inplace=True,drop=True)
# adult_valid_data.reset_index(inplace=True,drop=True)

In [12]:
# Torch defualt Dataloader
# N * 12 * 5000

# adult_train_dataset = CustomDataset(adult_file_path,adult_train_data)
# adult_valid_dataset = CustomDataset(adult_file_path,adult_valid_data)
# train_adult_loader = torch.utils.data.DataLoader(adult_train_dataset, batch_size = 32, shuffle=False) # len(adult_train_dataset)
# valid_adult_loader = torch.utils.data.DataLoader(adult_valid_dataset, batch_size = 32, shuffle=False) # len(adult_valid_dataset)

In [13]:
# Tensorflow default Dataloader

# train_adult_loader = TF_Dataloader(adult_file_path, adult_train_data, 32, shuffle=True)
# valid_adult_loader = TF_Dataloader(adult_file_path, adult_valid_data, 32, shuffle=False)

In [7]:
# Tensorflow Dataloader + ECG data normalized by mean and std
adult_file_path = './dataset/numpy_joined/'
train_adult_loader = ad_TF_Dataloader_Norm(adult_file_path, adult_train_, 32, shuffle=True)
valid_adult_loader = ad_TF_Dataloader_Norm(adult_file_path, adult_valid, 32, shuffle=False)

In [15]:
# (N*12*5000) -> (N*5000*12)

# train_batch = next(iter(train_adult_loader))
# valid_batch = next(iter(valid_adult_loader))
# train_input = train_batch[0].view(-1,12,5000).transpose(1,2).numpy()
# train_label = train_batch[2].numpy()
# valid_input = valid_batch[0].view(-1,12,5000).transpose(1,2).numpy()
# valid_label = valid_batch[2].numpy()

In [8]:
with tf.device('/device:CPU:0'):
    pre_trained_model = load_pretrained_model()
    latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
    adult_model = AppendNet(latent)
    adult_model.build((None,5000,12))
optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-4)
loss_fn = tf.keras.losses.MeanSquaredError()
epochs=20
with tf.device('/device:CPU:0'):
    adult_model.compile(optimizer, loss_fn, metrics = 'mae')
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./ckp/adult_weight_2',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [9]:
with tf.device('/device:CPU:0'):
    adult_model.fit(train_adult_loader, batch_size=32, epochs=epochs, validation_data=valid_adult_loader, callbacks=[model_checkpoint_callback],workers=4)

Epoch 1/20
3109/3109 [==============================] - ETA: 0s - loss: 487.9904 - mae: 16.5282

INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


3109/3109 [==============================] - 826s 265ms/step - loss: 487.9904 - mae: 16.5282 - val_loss: 151.0527 - val_mae: 10.0767
Epoch 2/20
3109/3109 [==============================] - ETA: 0s - loss: 365.6077 - mae: 14.8490

INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


...

INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


3109/3109 [==============================] - 885s 284ms/step - loss: 192.1330 - mae: 11.0146 - val_loss: 93.8927 - val_mae: 7.6332
Epoch 18/20
3109/3109 [==============================] - ETA: 0s - loss: 190.8719 - mae: 10.9813

INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


INFO:tensorflow:Assets written to: ./ckp\adult_weight_2\assets


3109/3109 [==============================] - 899s 289ms/step - loss: 190.8719 - mae: 10.9813 - val_loss: 93.0149 - val_mae: 7.6139
Epoch 19/20
3109/3109 [==============================] - 897s 288ms/step - loss: 193.8441 - mae: 10.9154 - val_loss: 94.3434 - val_mae: 7.6724
Epoch 20/20
3109/3109 [==============================] - 904s 291ms/step - loss: 183.0306 - mae: 10.7113 - val_loss: 95.7748 - val_mae: 7.7481


In [ ]:
# output = adult_model.predict(valid_input)
# output = output.flatten()

110/110 [==============================] - 6s 51ms/step


In [ ]:
# from sklearn.metrics import mean_absolute_error
# mean_absolute_error(output,valid_label)

8.897496129202317

# Child

In [10]:
child_train = pd.read_csv('./dataset/ECG_child_age_train_splited.csv')
child_valid = pd.read_csv('./dataset/ECG_child_age_valid_splited.csv')
new_train_child = pd.read_csv('./dataset/new_train_child.csv')
new_valid_child = pd.read_csv('./dataset/new_valid_child.csv')
child_train_ = pd.concat([child_train,new_train_child])
child_valid_ = pd.concat([child_valid,new_valid_child])
child_train_.reset_index(drop=True,inplace=True)
child_valid_.reset_index(drop=True,inplace=True)

In [11]:
# Tensorflow Dataloader + ECG data normalized by mean and std
child_file_path = './dataset/numpy_joined/'
train_child_loader = ad_TF_Dataloader_Norm(child_file_path, child_train_, 32, shuffle=True)
valid_child_loader = ad_TF_Dataloader_Norm(child_file_path, child_valid_, 32, shuffle=False)

In [ ]:
# train_batch = next(iter(train_child_loader))
# valid_batch = next(iter(valid_child_loader))
# train_input = train_batch[0].view(-1,12,5000).transpose(1,2).numpy()
# train_label = train_batch[2].numpy()
# valid_input = valid_batch[0].view(-1,12,5000).transpose(1,2).numpy()
# valid_label = valid_batch[2].numpy()

In [12]:
with tf.device('/device:CPU:0'):
    pre_trained_model = load_pretrained_model()
    latent = tf.keras.Model(pre_trained_model.inputs, pre_trained_model.get_layer('embed').output)
    child_model = AppendNet(latent)
    child_model.build((None,5000,12))
optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-4)
loss_fn = tf.keras.losses.MeanSquaredError()
epochs=10
with tf.device('/device:CPU:0'):
    child_model.compile(optimizer, loss_fn, metrics = 'mae')
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./ckp/child_weight',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [13]:
with tf.device('/device:CPU:0'):
    child_model.fit(train_child_loader, batch_size=32, epochs=epochs, validation_data=valid_child_loader, callbacks=[model_checkpoint_callback],workers=4)

Epoch 1/10
260/260 [==============================] - ETA: 0s - loss: 15.4189 - mae: 2.4982

INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


260/260 [==============================] - 85s 322ms/step - loss: 15.4189 - mae: 2.4982 - val_loss: 33.0917 - val_mae: 3.6696
Epoch 2/10
260/260 [==============================] - ETA: 0s - loss: 12.7268 - mae: 2.1737

INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


...

INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


260/260 [==============================] - 83s 318ms/step - loss: 9.8788 - mae: 1.8505 - val_loss: 23.0089 - val_mae: 3.0183
Epoch 7/10
260/260 [==============================] - ETA: 0s - loss: 8.7402 - mae: 1.7586

INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


INFO:tensorflow:Assets written to: ./ckp\child_weight\assets


260/260 [==============================] - 82s 316ms/step - loss: 8.7402 - mae: 1.7586 - val_loss: 20.8285 - val_mae: 2.8560
Epoch 8/10
260/260 [==============================] - 78s 300ms/step - loss: 7.6336 - mae: 1.6665 - val_loss: 23.6891 - val_mae: 2.9927
Epoch 9/10
260/260 [==============================] - 77s 298ms/step - loss: 6.4550 - mae: 1.5521 - val_loss: 21.0429 - val_mae: 3.1030
Epoch 10/10
260/260 [==============================] - 78s 301ms/step - loss: 5.5542 - mae: 1.4653 - val_loss: 25.0300 - val_mae: 3.0661


In [ ]:
# output = child_model.predict(valid_input)
# output = output.flatten()

28/28 [==============================] - 1s 51ms/step


In [ ]:
# from sklearn.metrics import mean_absolute_error
# mean_absolute_error(output,valid_label)

0.806895284435895

# inference

# Load Dataset

In [ ]:
submission = pd.read_csv('./dataset/submission.csv')
adult_file_path = './dataset/ECG_adult_numpy_valid/'
child_file_path = './dataset/ECG_child_numpy_valid/'

In [ ]:
submission['gap'] = submission['FILENAME'].apply(lambda x : x[4:9])
child_df = submission[submission['gap'] == 'child']
adult_df = submission[submission['gap'] == 'adult']
child_df.drop('gap', inplace=True, axis= 1)
adult_df.drop('gap', inplace=True, axis= 1)

C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18120\4184683789.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_df.drop('gap', inplace=True, axis= 1)
C:\Users\SNUH벤처\AppData\Local\Temp\ipykernel_18120\4184683789.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adult_df.drop('gap', inplace=True, axis= 1)


In [ ]:
adult_dataset = TestDataset(adult_file_path,adult_df)
child_dataset = TestDataset(child_file_path,child_df)
adult_loader = torch.utils.data.DataLoader(adult_dataset, batch_size = len(adult_dataset), shuffle=False)
child_loader = torch.utils.data.DataLoader(child_dataset, batch_size = len(child_dataset), shuffle=False)

In [ ]:
adult_batch = next(iter(adult_loader))
child_batch = next(iter(child_loader))
adult_input = adult_batch[0].view(-1,12,5000).transpose(1,2).numpy()
child_input = child_batch[0].view(-1,12,5000).transpose(1,2).numpy()

(60000,)